TASK #1 VOCALBULARY

In [1]:
import numpy as np

In [2]:
import pandas as pd
from google.colab import drive
drive.mount("/content/drive")
train=pd.read_csv('/content/drive/My Drive/Colab Notebooks/HW2-CSCI544/train', sep='\t', names = ['index', 'word', 'pos'])

Mounted at /content/drive


In [3]:
vocab_dict = train['word'].value_counts().to_dict()

In [4]:
vocabulary = pd.DataFrame({'word': ['<unk>'], 'index': [0], 'frequency': [0]})

In [5]:
count=0
for st, val in vocab_dict.items():
  if val>=3:
    count+=1
    new_row=pd.DataFrame({'word': [st], 'index': [count], 'frequency': [val]})
    vocabulary = pd.concat([vocabulary, new_row], axis=0)
  else:
    vocabulary.iloc[0,2] += val
vocabulary=vocabulary.reset_index(drop=True)

In [6]:
vocabulary

,word,index,frequency
0,<unk>,0,32537
1,",",1,46476
2,the,2,39533
3,.,3,37452
4,of,4,22104
...,...,...,...
16915,Bulgarian,16915,3
16916,markkaa,16916,3
16917,Therefore,16917,3
16918,injecting,16918,3


In [7]:
print("The threshold frequency for vocabulary is 3")

The threshold frequency for vocabulary is 3


In [8]:
vocabulary.to_csv('/content/vocab.txt',sep='\t')

TASK #2 PROBABILITIES

In [9]:
train['word_count'] = train.groupby('word')["word"].transform('size')

In [10]:
train['word'] = train.apply(lambda x : '<unk>' if x['word_count']<3 else x['word'], axis = 1)

In [11]:
df_pos_tag = train['pos'].value_counts().rename_axis('pos').reset_index(name = 'count')
pos_dict = dict(df_pos_tag.values)
pos_tags = df_pos_tag.pos.tolist()

In [12]:
train_sentences = []
train_sentence = []
first_sentence = 1
for train_row in train.itertuples():
    if(train_row.index == 1 and first_sentence == 0):
        train_sentences.append(train_sentence)
        train_sentence = []
    first_sentence = 0
    train_sentence.append((train_row.word, train_row.pos))
train_sentences.append(train_sentence)

In [13]:
def generate_start_probability(df, pos_tags):
    tags_start = {}
    total_tags_start = 0

    for tag in pos_tags:
      tags_start[tag]=0
    
    for df_row_tuple in df.itertuples():
        if(df_row_tuple.index == 1):
            tags_start[df_row_tuple.pos] += + 1
            total_tags_start += 1
    
    start_prob = {}
    for key in tags_start:
        start_prob[key] = tags_start[key] / total_tags_start
    
    return start_prob

In [14]:
def calculate_tp(pos_tags, sentences, start_prob):
  tp_matrix = np.zeros((len(pos_tags),len(pos_tags)))
  pos_tag_count_dict={}

  for sentence in sentences:
    for i in range(len(sentence)):
      pos_tag_count_dict[pos_tags.index(sentence[i][1])] = pos_tag_count_dict.get(pos_tags.index(sentence[i][1]), 0) + 1
      if i==0: continue
      tp_matrix[pos_tags.index(sentence[i - 1][1])][pos_tags.index(sentence[i][1])]+=1
  
  for i in range(len(pos_tags)):
    for j in range(len(pos_tags)):
       if tp_matrix[i][j]==0:
         tp_matrix[i][j]=1e-10
       else: 
         tp_matrix[i][j] = tp_matrix[i][j]/pos_tag_count_dict[i]
  
  pos_tags_dict=dict(enumerate(pos_tags))

  tp_dict={}

  for i in range(len(pos_tags_dict)):
    for j in range(len(pos_tags_dict)):
      tp_dict[pos_tags_dict[i]+" "+pos_tags_dict[j]] = tp_matrix[i][j]

  for i in range(len(pos_tags_dict)):
    tp_dict["<start>"+" "+pos_tags_dict[i]] = start_prob[pos_tags_dict[i]]

  return [tp_matrix, tp_dict]

In [15]:
def calculate_ep(pos_tags, words, sentences):
  ep_matrix = np.zeros((len(pos_tags), len(words)))
  pos_tag_count_dict={}

  for sentence in sentences:
    for word, pos_tag in sentence:
      pos_tag_count_dict[pos_tags.index(pos_tag)] = pos_tag_count_dict.get(pos_tags.index(pos_tag), 0) + 1
      ep_matrix[pos_tags.index(pos_tag)][words.index(word)] += 1
  # print(ep_matrix[])
  for i in range(len(pos_tags)):
    for j in range(len(words)):
      if ep_matrix[i][j]==0:
        ep_matrix[i][j]=1e-10
      else:
         ep_matrix[i][j] =  ep_matrix[i][j]/pos_tag_count_dict[i]
  print(ep_matrix[pos_tags.index("DT")][words.index("The")])

  pos_tags_dict=dict(enumerate(pos_tags))

  ep_dict={}

  for i in range(len(pos_tags_dict)):
    for j in range(len(words)):
      ep_dict[pos_tags_dict[i]+" "+words[j]] = ep_matrix[i][j] 

  return [ep_matrix, ep_dict]


In [16]:
start_prob = generate_start_probability(train, pos_dict)

In [17]:
tp_matrix, tp_dict = calculate_tp(pos_tags, train_sentences, start_prob)

In [18]:
ep_matrix, ep_dict= calculate_ep(pos_tags, vocabulary['word'].tolist(), train_sentences)

0.08304665185655348


In [19]:
len(ep_dict)

761400

In [20]:
len(tp_dict)

2070

In [21]:
import json
with open("/content/hmm.json", "w") as outfile:
    json.dump(tp_dict, outfile)
    outfile.write('\n')
    json.dump(ep_dict, outfile)

TASK #3 GREEDY DECODING

In [22]:
dev=pd.read_csv('/content/drive/My Drive/Colab Notebooks/HW2-CSCI544/dev', sep='\t', names = ['index', 'word', 'pos'])

In [23]:
dev_sentences = []
dev_sentence = []
first_sentence = 1
for dev_row in dev.itertuples():
    if(dev_row.index == 1 and first_sentence == 0):
        dev_sentences.append(dev_sentence)
        dev_sentence = []
    first_sentence = 0
    dev_sentence.append((dev_row.word, dev_row.pos))
dev_sentences.append(dev_sentence)

In [24]:
len(dev_sentences)

5527

In [25]:
dev

,index,word,pos
0,1,The,DT
1,2,Arizona,NNP
2,3,Corporations,NNP
3,4,Commission,NNP
4,5,authorized,VBD
...,...,...,...
131763,13,join,VB
131764,14,the,DT
131765,15,winning,VBG
131766,16,bidder,NN


In [26]:
def greedy(tp_dict, ep_dict, start_prob, sentences, pos_tags):
  predicted_tags=[]
  predicted_scores=[]

  for sentence in sentences:
    prev=None
    sentence_tags=[]
    sentence_scores=[]

    for word_index in range(len(sentence)):
      max=-1

      for i, tag in enumerate(pos_tags):
        result=1

        if str(tag+" "+sentence[word_index][0]) in ep_dict:
          result*=ep_dict[tag+" "+sentence[word_index][0]]
        else:
          result*=ep_dict[tag+" "+"<unk>"]

        if word_index!=0:
          if str(prev+" "+tag) in tp_dict:
            result*=tp_dict[prev+" "+tag]
        else:
          result*=start_prob[tag]

        if(result>max):
          max=result
          temp=tag

      prev=temp
      sentence_tags.append(prev)
      sentence_scores.append(max)

    predicted_tags.append(sentence_tags)
    predicted_scores.append(sentence_scores)

  return predicted_tags, predicted_scores

In [27]:
def accuracy(original_pos, predicted_pos):
  count=0
  count_tag=0

  for i in range(len(original_pos)):
    for j in range(len(original_pos[i])):
      if original_pos[i][j][1] == predicted_pos[i][j]:
        count_tag+=1
      count+=1
  return ((count_tag*100)/count)

In [28]:
predicted_tags, predicted_scores = greedy(tp_dict, ep_dict, start_prob, dev_sentences, pos_tags)

In [29]:
print("Accuracy for greedy algorithm is " + str(accuracy(dev_sentences, predicted_tags)))

Accuracy for greedy algorithm is 92.99450549450549


TASK #4 VITERBI DECODING

In [30]:
def viterbi(tp_dict, ep_dict, start_prob, sentences, pos_tags):
  
  dp_matrix=[]
  predicted_tags_dict=[]

  for sentence in sentences:
    n = len(pos_tags)
    dp_probab = []
    decode_string = {}
    for tag in pos_tags:
      if str(tag + " " + sentence[0][0]) in ep_dict:
        dp_probab.append(start_prob[tag] * ep_dict[tag + " " + sentence[0][0]])
      else:
        dp_probab.append(start_prob[tag] * ep_dict[tag + " " + "<unk>"])
    for i, l in enumerate(sentence):
      word = l[0]
      if i == 0: continue
      temp_list = [None] * n
      for j,tag in enumerate(pos_tags):
        max = -1
        result = 1
        for k, prob in enumerate(dp_probab):
          if str(pos_tags[k] + " " + tag) in tp_dict and str(tag + " " + word) in ep_dict:
            result = prob * tp_dict[pos_tags[k] + " " + tag] * ep_dict[tag + " " + word]
          else:
            result = prob * tp_dict[pos_tags[k] + " " + tag] * ep_dict[ tag + " " + "<unk>"]
          if(max < result):
            max = result
            decode_string[str(i) + " " + tag] = [pos_tags[k], result]
        temp_list[j] = max
      dp_probab = [x for x in temp_list]
    dp_matrix.append(dp_probab)
    predicted_tags_dict.append(decode_string)

  predicted_tags=[]
  predicted_tags_scores=[]
  for decode_string, dp_probab in zip(predicted_tags_dict, dp_matrix):
    no_of_tags = len(pos_tags)
    n = len(decode_string) // no_of_tags
    predicted_tag = []
    score = []
    x = pos_tags[np.argmax(np.asarray(dp_probab))]
    predicted_tag.append(x)
    for i in range(n, 0, -1):
        val = decode_string[str(i) + ' ' + x][1]
        x = decode_string[str(i) + ' ' + x][0]
        predicted_tag = [x] + predicted_tag
        score =  [val] + score
    predicted_tags.append(predicted_tag)
    predicted_tags_scores.append(score)
  return predicted_tags, predicted_tags_scores

In [31]:
predicted_tags, predicted_tags_scores = viterbi(tp_dict, ep_dict, start_prob, dev_sentences, pos_tags)

In [32]:
print("Accuracy for viterbi algorithm is " + str(accuracy(dev_sentences, predicted_tags)))

Accuracy for viterbi algorithm is 94.40835407686237


TEST OUTPUTS

In [33]:
test=pd.read_csv('/content/drive/My Drive/Colab Notebooks/HW2-CSCI544/test', sep='\t', names = ['index', 'word'] )

In [34]:
test_sentences = []
test_sentence = []
ignore_first_sentence=1

for _ in test.itertuples():
  if _.index==1:
    if ignore_first_sentence==0:
      test_sentences.append(test_sentence)
      test_sentence = []
  ignore_first_sentence=0
  test_sentence.append((_.word))

test_sentences.append(test_sentence)

In [35]:
test_greedy_tags, test_greedy_scores = greedy(tp_dict, ep_dict, start_prob, test_sentences, pos_tags)

In [36]:
def output_file(test_inputs, test_outputs):
    res = []
    for i in range(len(test_inputs)):
        s = []
        for j in range(len(test_inputs[i])):
            s.append((str(j+1), test_inputs[i][j], test_outputs[i][j]))
        res.append(s)
    
    with open("greedy.out", 'w') as f:
        for ele in res:
            f.write("\n".join([str(item[0]) + "\t" + item[1] + "\t" + item[2] for item in ele]))
            f.write("\n\n")

In [37]:
output_file(test_sentences, test_greedy_tags)

In [38]:
def output_file(test_inputs, test_outputs):
    res = []
    for i in range(len(test_inputs)):
        s = []
        for j in range(len(test_inputs[i])):
            s.append((str(j+1), test_inputs[i][j], test_outputs[i][j]))
        res.append(s)
    
    with open("viterbi.out", 'w') as f:
        for ele in res:
            f.write("\n".join([str(item[0]) + "\t" + item[1] + "\t" + item[2] for item in ele]))
            f.write("\n\n")

In [39]:
test_viterbi_tags, test_viterbi_scores = viterbi(tp_dict, ep_dict, start_prob, test_sentences, pos_tags)

In [40]:
output_file(test_sentences, test_viterbi_tags)